In [5]:
from keras.models import Model 
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [6]:
poor = pd.read_csv("SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
bank_reserves_std = []
for i in np.arange(0, poor.shape[0]):
    if(i % 500==0):
        print("On iteration ", i, "at ", datetime.datetime.now())
    sample = pd.concat((poor.iloc[i], middle.iloc[i]), axis=1)
    sample = pd.concat((sample, rich.iloc[i]), axis=1)
    sample_std = sc.fit_transform(sample)
    bank_reserves.append(sample)
    bank_reserves_std.append(sample_std)

On iteration  0 at  2024-10-01 15:57:15.161569
On iteration  500 at  2024-10-01 15:57:16.161870
On iteration  1000 at  2024-10-01 15:57:17.112886
On iteration  1500 at  2024-10-01 15:57:18.174402
On iteration  2000 at  2024-10-01 15:57:19.133358
On iteration  2500 at  2024-10-01 15:57:20.129918
On iteration  3000 at  2024-10-01 15:57:21.104867
On iteration  3500 at  2024-10-01 15:57:22.059973
On iteration  4000 at  2024-10-01 15:57:23.021414
On iteration  4500 at  2024-10-01 15:57:23.986313
On iteration  5000 at  2024-10-01 15:57:25.070307
On iteration  5500 at  2024-10-01 15:57:26.067705
On iteration  6000 at  2024-10-01 15:57:27.099285
On iteration  6500 at  2024-10-01 15:57:28.108124
On iteration  7000 at  2024-10-01 15:57:29.171131
On iteration  7500 at  2024-10-01 15:57:30.200766
On iteration  8000 at  2024-10-01 15:57:31.212666
On iteration  8500 at  2024-10-01 15:57:32.375323
On iteration  9000 at  2024-10-01 15:57:33.377929
On iteration  9500 at  2024-10-01 15:57:34.438359
On i

In [26]:
input = Input(shape=(303,))

encoded = Conv1D(filters=10, kernel_size=32, activation='relu')(input)
encoded = Conv1D(filters=10, kernel_size=64, activation='relu')(encoded)
encoded = Conv1D(filters=10, kernel_size=128, activation='relu')(encoded)
decoded = Conv1D(filters=10, kernel_size=128, activation='relu')(encoded)

encoded = Dense(384, activation='relu')(encoded)
decoded = Dense(5, activation='relu')(encoded)
decoded = Dense(384, activation='relu')(decoded)

decoded = Conv1DTranspose(filters=10, kernel_size=128, activation='relu')(decoded)
decoded = Conv1DTranspose(filters=10, kernel_size=128, activation='relu')(decoded)
decoded = Conv1DTranspose(filters=10, kernel_size=64, activation='relu')(decoded)
decoded = Conv1DTranspose(filters=10, kernel_size=32, activation='relu')(decoded)

autoencoder = Model(input, decoded)
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 30)             │        23,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 20)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 30)             │           630 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 30)             │           930 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,270 (102.62 KB)

 Trainable params: 26,270 (102.62 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [18]:
autoencoder.fit(x_train, x_train,
 epochs=20,
 batch_size=256,
 shuffle=True,
 validation_data=(x_test, x_test))

Epoch 1/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3400 - val_loss: 0.1673
Epoch 2/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1591 - val_loss: 0.1358
Epoch 3/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1331 - val_loss: 0.1230
Epoch 4/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1226 - val_loss: 0.1163
Epoch 5/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1163 - val_loss: 0.1110
Epoch 6/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1116 - val_loss: 0.1084
Epoch 7/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1087 - val_loss: 0.1058
Epoch 8/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1061 - val_loss: 0.1031
Epoch 9/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1037 - val_loss: 0.1008
Epoch 10/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1013 - val_loss: 0.0992
Epoch 11/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0997 - val_loss: 0.0975
Epoch 12/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

In [19]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = autoencoder.predict(x_test)
decoded_imgs = autoencoder.predict(encoded_imgs)
encoded_imgs[0].shape

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step


(784,)

In [25]:
encoded_imgs[0]

array([5.41694033e-12, 2.88966686e-13, 1.32832677e-12, 3.21480174e-13,
       1.90982827e-10, 3.52696455e-14, 1.04270488e-12, 7.03432364e-12,
       1.09835177e-12, 5.80059742e-12, 2.52220093e-13, 2.20588283e-12,
       7.89207755e-10, 1.40338327e-12, 1.89377001e-14, 2.44469115e-12,
       8.26794796e-12, 4.62759334e-13, 2.51384174e-11, 2.12008622e-12,
       1.07103718e-13, 2.13619417e-11, 4.78587656e-13, 4.87897873e-12,
       1.86328817e-12, 1.78015404e-12, 5.38730649e-12, 5.34545010e-13,
       4.89750168e-13, 3.34637401e-13, 1.79254506e-13, 2.12937892e-12,
       1.06172529e-10, 8.49495225e-13, 3.07269654e-10, 7.38391848e-10,
       1.71481296e-09, 1.87491017e-12, 1.99505239e-11, 1.19663862e-10,
       2.24297642e-12, 3.34000168e-14, 1.79949221e-17, 2.42300330e-19,
       4.14213107e-17, 2.19781289e-18, 2.84282517e-18, 1.10552193e-18,
       7.79775487e-17, 1.57692381e-13, 1.60540192e-10, 3.55180285e-09,
       1.42398402e-11, 5.09821298e-13, 1.10856788e-12, 2.12619037e-11,
      

In [20]:
decoded_imgs[0].shape

(784,)

In [23]:
# use Matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline = 10 # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
 # display original
 ax = plt.subplot(2, n, i + 1)
 plt.imshow(x_test[i].reshape(28, 28))
 plt.gray()
 ax.get_xaxis().set_visible(False)
 ax.get_yaxis().set_visible(False)# display reconstruction
 ax = plt.subplot(2, n, i + 1 + n)
 plt.imshow(decoded_imgs[i].reshape(28, 28))
 plt.gray()
 ax.get_xaxis().set_visible(False)
 ax.get_yaxis().set_visible(False)
plt.show()

NameError: name 'n' is not defined

<Figure size 2000x400 with 0 Axes>